In [ ]:
import pandas as pd
from pathlib import Path
from collections import defaultdict, Counter
from collections import OrderedDict
import warnings

In [35]:
warnings.simplefilter("ignore", UserWarning)

repo_root = Path().cwd()
print(repo_root)
print(repo_root)

ref_path  = repo_root / "master_sheet_reference.xlsx"
ag_folder    = repo_root / "assessment_guide"
ag_files  = sorted(ag_folder.glob("*.xls*"))

print("Found workbooks:")
for p in ag_files:
    print("•", p.name)

ref_sheets  = pd.read_excel(ref_path,  sheet_name=None)
print("Reference file sheets:", list(ref_sheets.keys()))

c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\lighthouse_dash
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\lighthouse_dash
Found workbooks:
• assessment_guide_diriyah.xlsx
• assessment_guide_qiddiyah.xlsm
Reference file sheets: ['P&M Schedule', 'P&M RIsk', 'Cost Estimation & Optimization', 'D&C - Quality', 'D&C - Procurement', 'D&C - HSE & Welfare', 'Innovation & Technology', 'Design & Technical', 'D&C - Construction', 'Strategy & Operations', 'Visuals Menu']


In [29]:
# Unpack reference file
ref_tabs = []     

for sheet_name, df in ref_sheets.items():
    if {"Table", "Column"}.issubset(df.columns): # Check if columns titled "Table" and "Column" exist
        table_cols = (
            df.groupby("Table")["Column"]
              .apply(list)
              .to_dict()
        )
        ref_tabs.append({ sheet_name: table_cols })
    else:
        print(f"Skipping sheet {sheet_name!r}: missing \"Table\" and \"Columm\" columns")

ref_tabs

Skipping sheet 'Visuals Menu': missing "Table" and "Columm" columns


[{'P&M Schedule': {'AddDataPoint(P&M)': ['Additional Data Point',
    'Value',
    'Year',
    'Value Completed (Baseline) in M SAR',
    'Value Completed (Actual) in M SAR',
    'Value Completed (Forecast) in M SAR'],
   'DevCoAssessmentAnalysis(P&M)': ['Assessment Criteria', 'Value']}},
 {'P&M RIsk': {'AddDataPoint(P&M)': ['Additional Data Point',
    'Value',
    'Risk Category',
    'Number of Risk Items',
    'Risk',
    'Source',
    'Potential Impact',
    'Risk Rating',
    'Risk Status',
    'Current Measures'],
   'DevCoAssessmentAnalysis(P&M)': ['Assessment Criteria',
    'Value',
    'Rating']}},
 {'Cost Estimation & Optimization': {'AddDataPoint(CE&O)': ['Additional Data Point',
    'Value',
    'Year',
    'Budget Value (Baseline) in M SAR',
    'Budget Value (Revised) in M SAR',
    'Budget Value (Forecast) in M SAR'],
   'DevCoAssessmentAnalysis(CE&O)': ['Assessment Criteria', 'Value', 'Rating'],
   'DevCoAssessmentInput(CE&O)': ['Data Point', 'Input Value']}},
 {'D&C -

In [30]:
table_cols = defaultdict(list)

for tab_dict in ref_tabs:
    sheet_map = next(iter(tab_dict.values()))
    for table_name, cols in sheet_map.items():
        table_cols[table_name].extend(cols)
        
for table_name, all_cols in table_cols.items():
    counts = Counter(all_cols)
    dupes = [col for col, n in counts.items() if n > 1]
    if dupes:
        print(f"⚠️ Table “{table_name}” has duplicate columns: {dupes}")
    else:
        print(f"✅ Table “{table_name}” has no repeated columns.")

for table_name, cols in table_cols.items():
    table_cols[table_name] = list(OrderedDict.fromkeys(cols))

print(table_cols)

⚠️ Table “AddDataPoint(P&M)” has duplicate columns: ['Additional Data Point', 'Value']
⚠️ Table “DevCoAssessmentAnalysis(P&M)” has duplicate columns: ['Assessment Criteria', 'Value']
✅ Table “AddDataPoint(CE&O)” has no repeated columns.
✅ Table “DevCoAssessmentAnalysis(CE&O)” has no repeated columns.
✅ Table “DevCoAssessmentInput(CE&O)” has no repeated columns.
⚠️ Table “AddDataPoint(D&C)” has duplicate columns: ['Additional Data Point', 'Value']
⚠️ Table “DevCoAssessmentAnalysis(D&C)” has duplicate columns: ['Assessment Criteria', 'Value']
⚠️ Table “DevCoAssessmentInput(D&C)” has duplicate columns: ['Assessment Criteria', 'Data Point', 'Input Value']
✅ Table “AddDataPoint(I&T)” has no repeated columns.
✅ Table “DevCoAssessmentAnalysis(I&T)” has no repeated columns.
defaultdict(<class 'list'>, {'AddDataPoint(P&M)': ['Additional Data Point', 'Value', 'Year', 'Value Completed (Baseline) in M SAR', 'Value Completed (Actual) in M SAR', 'Value Completed (Forecast) in M SAR', 'Risk Category'

In [ ]:
file_frames = []   # to collect df for each file

for data_path in ag_files:
    # derive suffix
    parts = data_path.stem.split("assessment_guide_")
    suffix = parts[1]

    print(f"Reading {data_path.name!r} (suffix = {suffix})")

    pieces = []
    for table_name, cols in table_cols.items():
        df = pd.read_excel(data_path, sheet_name=table_name, header=4)  # pull in AG data. Each table name from column reference sheet is a sheet name in the AG file.
        df.columns = df.columns.str.strip()

        present = [c for c in cols if c in df.columns] # cols = columns from reference, present = columns in AG
        missing = set(cols) - set(present)
        if missing:
            print(f"⚠️ In table {table_name!r}, missing columns: {missing}")

        df_sub = df[present].copy()
        for c in missing:
            df_sub[c] = pd.NA

        pieces.append(df_sub)

    file_df = pd.concat(pieces, axis=1) # master df for each file

    file_df.insert(0, "DevCo", [suffix] * len(file_df)) # append DevCo column

    file_frames.append(file_df) # append master df for each file to a list

master_df = pd.concat(file_frames, axis=0, ignore_index=True) # stack master df for each file row-wise

print("Final master shape:", master_df.shape)
master_df

Reading 'assessment_guide_diriyah.xlsx' (suffix = diriyah)
⚠️ In table 'AddDataPoint(D&C)', missing columns: {nan}
Reading 'assessment_guide_qiddiyah.xlsm' (suffix = qiddiyah)
⚠️ In table 'AddDataPoint(D&C)', missing columns: {nan}
Final master shape: (156, 67)


,DevCo,Additional Data Point,Value,Year,Value Completed (Baseline) in M SAR,Value Completed (Actual) in M SAR,Value Completed (Forecast) in M SAR,Risk Category,Number of Risk Items,Risk,...,Data Point,Input Value,Additional Data Point,Value,Assessment Criteria,Value,Rating,Performance Signal Score,Performance Signal,SourceFile
0,diriyah,Latest Approved Completion Date​,Q2-28,2025.0,0.0,NaN,NaN,NaN,NaN,Risk 1,...,Environmental mitigation measures included in ...,0.1,Implementation Success Rate for Innovation Str...,3.0,Is there an innovation Strategy or document th...,0.4,Needs Improvement,1.7,Innovation & Technology Deployment,assessment_guide_diriyah.xlsx
1,diriyah,Forecast Completion Date​,Q1-29,2026.0,1250.0,1250.0,NaN,NaN,NaN,Risk 2,...,Environmental mitigation measures recommended ...,0.2,Innovation Adoption Effectiveness Index (strat...,1.0,NaN,NaN,NaN,NaN,NaN,assessment_guide_diriyah.xlsx
2,diriyah,Approved baseline completion date,Q3-27,2027.0,2000.0,1550.0,1550.0,Risk 1,19.0,Risk 3,...,Update of construction execution plan,Yes,Innovation Adoption Effectiveness Index (skill...,3.0,NaN,NaN,NaN,NaN,NaN,assessment_guide_diriyah.xlsx
3,diriyah,No. of schedule iterations,8,2028.0,3100.0,NaN,2200.0,Risk 2,16.0,Risk 4,...,Review and approval of Construction sequence &...,No,Innovation Adoption Effectiveness Index (plann...,2.0,NaN,NaN,NaN,NaN,NaN,assessment_guide_diriyah.xlsx
4,diriyah,Planned duration up to a specific phase as per...,120,2029.0,NaN,NaN,3100.0,Risk 3,12.0,Risk 5,...,"Planned Area provision for site offices, ameni...",Average,Innovation Adoption Effectiveness Index (proce...,4.0,NaN,NaN,NaN,NaN,NaN,assessment_guide_diriyah.xlsx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,qiddiyah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Total number of Closed Welfare Observations,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,assessment_guide_qiddiyah.xlsm
152,qiddiyah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Total Welfare Non-compliances,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,assessment_guide_qiddiyah.xlsm
153,qiddiyah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Total Closed Welfare Non-compliances,58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,assessment_guide_qiddiyah.xlsm
154,qiddiyah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Cycle time to close Welfare Non-Compliances,10 Days,NaN,NaN,NaN,NaN,NaN,NaN,NaN,assessment_guide_qiddiyah.xlsm


In [34]:
output_path = repo_root/ 'master_sheet.xlsx'
master_df.to_excel(output_path, index=False)

print(f"✅ Saved master sheet to {output_path!r}")

✅ Saved master sheet to WindowsPath('c:/Users/jeremy.yap/OneDrive - Meinhardt Singapore Pte Ltd/Python/lighthouse_dash/master_sheet.xlsx')
